In [10]:
from mysql_connection import get_cursor
import time
import pandas as pd
import numpy as np

cursor = get_cursor()
# Start the timer
start = time.perf_counter()

## Call get food checks by id with the user id

In [11]:
user_id = (1,)
foodchecks = cursor.callproc("get_foodchecks_by_id", user_id)
checks = []
# stored_results is iterable, have to do this way
for result in cursor.stored_results():
    checks = result.fetchall()[0][1:]

print(checks)

(0, 0, 0, 0, 1, 1, 1, 1, 1)


## Parse all the results, and create the WHERE clause

In [12]:
column_rows = ["IsVegetarian", "IsVegan", "IsHalal", "IsKosher", "HasLactose", "HasNuts", "HasGluten", "HasEgg", "HasSoy"]

def print_food_checks(result) :
    for i in range(len(result)):
        print(f"{column_rows[i]} : {result[i]}")

print_food_checks(checks)

def food_checks_builder(result):
    food_checks = []
    for i in range(len(result)):
        if(result[i]):
            food_checks.append(f"{column_rows[i]}=1")
    return " AND ".join(food_checks)

print("---------------------------------")
print(food_checks_builder(checks))

IsVegetarian : 0
IsVegan : 0
IsHalal : 0
IsKosher : 0
HasLactose : 1
HasNuts : 1
HasGluten : 1
HasEgg : 1
HasSoy : 1
---------------------------------
HasLactose=1 AND HasNuts=1 AND HasGluten=1 AND HasEgg=1 AND HasSoy=1


## Get all the user id's with at least the food checks that you have
### i.e. You are vegetarian... Some one who is vegetarian, egg-free, and lactose-free will also be selected 

In [13]:
cursor.execute(f"SELECT UserID, FoodTagID, RightSwipePercent FROM FoodOpinionRightSwipePercent WHERE UserID IN(SELECT UserID FROM tat.UserFoodCheckView WHERE {food_checks_builder(checks)});")
result = cursor.fetchall()

## Convert the result into a dataframe, and select the top 5 most liked and disliked items


In [14]:
from user import User
NUMBER_OF_ITEMS_TO_GET = 5

df = pd.DataFrame(result)
df.columns = ["UserID", "FoodTagID", "RightSwipePercent"]
pointer = 0
data = []

while pointer < len(df):
    user_indexes = df["UserID"][df["UserID"] == df["UserID"][pointer]].index
    first_occurance, last_occurance = user_indexes[0], user_indexes[-1]

    # If the user hasn't swiped on atleast 10 items, skip them

    if len(user_indexes) < 10:
        # The index after last_occurance will be the next item
        pointer = last_occurance + 1
    else:
        most_liked = df[first_occurance : first_occurance + NUMBER_OF_ITEMS_TO_GET]
        most_disliked = df[last_occurance - NUMBER_OF_ITEMS_TO_GET : last_occurance ]
        new_user = User(most_liked, most_disliked, False)
        data.append(new_user)
        pointer = last_occurance + 1
    
df.head()

,UserID,FoodTagID,RightSwipePercent
0,1,4,1.8649
1,1,9,1.8462
2,1,2,1.7647
3,1,1,1.6667
4,1,7,1.5789


## Time the exectution

In [15]:
end = time.perf_counter()
print(f"Done in {end - start:0.4f} seconds")

Done in 0.1275 seconds


In [16]:
User.help()

top_five_(dis)likes: tuple ( FoodTagID, RightSwipePercent )


## Print the users

In [17]:
for user in data:
    print("-----------------")
    print(f"Most liked: \n{user.top_five_likes}")
    print(f"Most disliked: \n{user.top_five_dislikes}")
    print("-----------------")

-----------------
Most liked: 
[(4, 1.8649), (9, 1.8462), (2, 1.7647), (1, 1.6667), (7, 1.5789)]
Most disliked: 
[(6, 1.0909), (8, 1.0), (5, 0.8889), (11, 0.25), (3, 0.0)]
-----------------
-----------------
Most liked: 
[(7, 1.9806), (2, 1.7647), (9, 1.7391), (1, 1.6667), (6, 1.0769)]
Most disliked: 
[(12, 0.7317), (4, 0.5455), (5, 0.5), (3, 0.4), (11, 0.3636)]
-----------------


## See how much memory the object takes up
### Use for any optimisations

In [18]:
from pympler import asizeof
print(asizeof.asizeof(data))


3384
